<a href="https://colab.research.google.com/github/vinay10949/AnalyticsAndML/blob/master/1_19_Missing_Category_Imputation_Feature_Engine.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Missing Category Imputation ==> Feature-Engine


### What is Feature-Engine

Feature-Engine is an open source python package that I created at the back of this course. 

- Feature-Engine includes all the feature engineering techniques described in the course
- Feature-Engine works like to Scikit-learn, so it is easy to learn
- Feature-Engine allows you to implement specific engineering steps to specific feature subsets
- Feature-Engine can be integrated with the Scikit-learn pipeline allowing for smooth model building
- 
**Feature-Engine allows you to design and store a feature engineering pipeline with bespoke procedures for different variable groups.**

-------------------------------------------------------------------
Feature-Engine can be installed via pip ==> pip install feature-engine

- Make sure you have installed feature-engine before running this notebook


In [0]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

# to split the datasets
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline

# from feature-engine
from feature_engine import missing_data_imputers as mdi

In [0]:
# let's load the dataset with a selected group of variables

cols_to_use = [
    'BsmtQual', 'FireplaceQu', 'LotFrontage', 'MasVnrArea', 'GarageYrBlt',
    'SalePrice'
]

data = pd.read_csv('../houseprice.csv', usecols=cols_to_use)
data.head()

,LotFrontage,MasVnrArea,BsmtQual,FireplaceQu,GarageYrBlt,SalePrice
0,65.0,196.0,Gd,NaN,2003.0,208500
1,80.0,0.0,Gd,TA,1976.0,181500
2,68.0,162.0,Gd,TA,2001.0,223500
3,60.0,0.0,TA,Gd,1998.0,140000
4,84.0,350.0,Gd,TA,2000.0,250000


In [0]:
data.isnull().mean()

LotFrontage    0.177397
MasVnrArea     0.005479
BsmtQual       0.025342
FireplaceQu    0.472603
GarageYrBlt    0.055479
SalePrice      0.000000
dtype: float64

In [0]:
# let's separate into training and testing set

# first drop the target from the feature list
cols_to_use.remove('SalePrice')

X_train, X_test, y_train, y_test = train_test_split(data[cols_to_use],
                                                    data['SalePrice'],
                                                    test_size=0.3,
                                                    random_state=0)
X_train.shape, X_test.shape

((1022, 5), (438, 5))

### Feature-Engine captures the categorical variables automatically

In [0]:
# we call the imputer from featur- engine
# we don't need to specify anything 

imputer = mdi.CategoricalVariableImputer()

In [0]:
# we fit the imputer

imputer.fit(X_train)

CategoricalVariableImputer(variables=['BsmtQual', 'FireplaceQu'])

In [0]:
# we see that the imputer found the categorical variables to
# impute with the frequent category

imputer.variables

['BsmtQual', 'FireplaceQu']

**This imputer will replace missing data in categorical variables by 'Missing'**

In [0]:
# feature-engine returns a dataframe

tmp = imputer.transform(X_train)
tmp.head()

,BsmtQual,FireplaceQu,LotFrontage,MasVnrArea,GarageYrBlt
64,Gd,Missing,NaN,573.0,1998.0
682,Gd,Gd,NaN,0.0,1996.0
960,TA,Missing,50.0,0.0,NaN
1384,TA,Missing,60.0,0.0,1939.0
1100,TA,Missing,60.0,0.0,1930.0


In [0]:
# let's check that the numerical variables don't
# contain NA any more

tmp[imputer.variables].isnull().mean()

BsmtQual       0.0
FireplaceQu    0.0
dtype: float64

## Feature-engine allows you to specify variable groups easily

In [0]:
# let's do it imputation but this time
# and let's do it over 1 of the 2 categorical variables

imputer = mdi.CategoricalVariableImputer(variables=['BsmtQual'])

imputer.fit(X_train)

CategoricalVariableImputer(variables=['BsmtQual'])

In [0]:
# now the imputer uses only the variables we indicated

imputer.variables

['BsmtQual']

In [0]:
# now the imputer uses only the variables we indicated

imputer.variables

['BsmtQual']

## Feature-engine can be used with the Scikit-learn pipeline

In [0]:
# let's check the percentage of NA in each categorical variable

X_train.isnull().mean()

BsmtQual       0.023483
FireplaceQu    0.467710
LotFrontage    0.184932
MasVnrArea     0.004892
GarageYrBlt    0.052838
dtype: float64

- BsmtQual: 0.023 ==> frequent category imputation
- FirePlaceQu: 0.46 ==> missing category imputation

In [0]:
pipe = Pipeline([
    ('imputer_mode', mdi.FrequentCategoryImputer(variables=['BsmtQual'])),
    ('imputer_missing', mdi.CategoricalVariableImputer(variables=['FireplaceQu'])),
])

In [0]:
pipe.fit(X_train)

Pipeline(memory=None,
     steps=[('imputer_mode', FrequentCategoryImputer(variables=['BsmtQual'])), ('imputer_missing', CategoricalVariableImputer(variables=['FireplaceQu']))])

In [0]:
pipe.named_steps['imputer_mode'].variables

['BsmtQual']

In [0]:
pipe.named_steps['imputer_missing'].variables

['FireplaceQu']

In [0]:
# let's transform the data with the pipeline
tmp = pipe.transform(X_train)

# let's check null values are gone
tmp.isnull().mean()

BsmtQual       0.000000
FireplaceQu    0.000000
LotFrontage    0.184932
MasVnrArea     0.004892
GarageYrBlt    0.052838
dtype: float64